In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-959fhd7u/unsloth_23b461c01e92474ea8cd845d8f9d47a8
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-959fhd7u/unsloth_23b461c01e92474ea8cd845d8f9d47a8
  Resolved https://github.com/unslothai/unsloth.git to commit a7bfbe7927ea75f959e1d7c84e7bf50945d405ff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.9 MB/s eta 0:00:00
   ━━

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 0,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
text_prompt = """ Solve this math question
Question: {}
Answer: {}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = text_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("juancopi81/orca-math-word-problems-190038_200035", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split:   0%|          | 0/9997 [00:00<?, ? examples/s]

Map:   0%|          | 0/9997 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "math-llama3",
        push_to_hub=True
    ),
)

Map (num_proc=2):   0%|          | 0/9997 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,997 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.894700
2,0.795600
3,0.745500
4,0.717800
5,0.996600
6,0.650400
7,0.664100
8,0.553200
9,0.541000
10,0.668500


In [ ]:
model.push_to_hub_merged("Vasanth/math-llama3", tokenizer, save_method = "merged_16bit")

In [ ]:
from peft import AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained("Vasanth/math-llama3")
model.merge_and_unload()
model.push_to_hub("Vasanth/math-llama3-nolora")

In [ ]:
from unsloth import FastLanguageModel

import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Vasanth/math-llama3-nolora",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
    )

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# model.push_to_hub("Vasanth/math-llama3")
# model.push_to_hub_merged("Vasanth/math-llama3", tokenizer, save_method = "merged_16bit")
model.push_to_hub_gguf("Vasanth/math-llama3-gguf", tokenizer, quantization_method = "q4_k_m")
# tokenizer.push_to_hub("Vasanth/math-llama3")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.22 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:41<00:00,  3.16s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving Vasanth/math-llama3-gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Vasanth/math-llama3-gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Vasanth/math-llama3-gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Vasanth/math-llama3-gguf/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at Vasanth/math-llama3-gguf into f16 GGUF format.
The output location will be ./Vasanth/math-llama3-gguf/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: math-llama3-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 128256}
INFO:hf-to-g

In [ ]:
FastLanguageModel.for_inference(model)
text_prompt = """ Solve this math question
Question: {}
Answer: {}
"""
inputs = tokenizer(
[
    text_prompt.format(
        "I have two apples. My father gave me two. How many do I have?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|> Solve this math question\nQuestion: I have two apples. My father gave me two. How many do I have?\nAnswer: \nYou initially have two apples. Your father gave you two more, so you now have:\n\n2 (initial apples) + 2 (apples from your father) = 4 apples\n\nYou now have 4 apples.\n<|eot_id|>']

In [ ]:
inputs = tokenizer(
[
    text_prompt.format(
        "I have two apples. My father gave me two. How many do I have?",
        "",
    )
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|> Solve this math question
Question: I have two apples. My father gave me two. How many do I have?
Answer: 
You started with two apples, and your father gave you two more. So, you now have a total of:

2 (initial apples) + 2 (apples from your father) = 4 apples

You have four apples now.
<|eot_id|>


In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
        "Vasanth/math-llama3",
        load_in_4bit = True,
    )
tokenizer = AutoTokenizer.from_pretrained("Vasanth/math-llama3")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
model.push_to_hub("Vasanth/math-llama3-nolora")

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Vasanth/math-llama3-nolora/commit/79cc348378740009194f8313ec9bb755bbaa43b4', commit_message='Upload LlamaForCausalLM', commit_description='', oid='79cc348378740009194f8313ec9bb755bbaa43b4', pr_url=None, pr_revision=None, pr_num=None)